### Producing Sentiment variations using LLMs

Author: Raphael Merx
Input: a baseline sentence; Output: variations of this sentence where a target word is more or less intense

In [1]:
#pip install simplemind python-dotenv

In [2]:
from dotenv import load_dotenv
load_dotenv()

from dataclasses import dataclass
from tqdm import tqdm
import simplemind as sm
from typing import Literal, List
import pandas as pd
import os
import random

random.seed(42)

TARGET_WORD_CHOICES = Literal['abuse', 'anxiety', 'depression', 'mental_health', 'mental_illness', 'trauma']

TARGET_WORD: TARGET_WORD_CHOICES = 'mental_illness'
TARGET_WORD_HUMAN = TARGET_WORD.replace('_', ' ')

MAX_BASELINES = 10

# can be changed to gemini, see https://pypi.org/project/simplemind/
PROVIDER = "openai"
MODEL = "gpt-4o"

Get neutral sentences from corpus for LLM input for each target

**Aim**: This script processes text files containing sentences for specified target terms, calculates sentence-level mean arousal scores using the NRC-VAD lexicon, identifies 
sentences with neutral arousal scores (dynamically globally determined from NRC-VAD dataset), and saves these sentences along with their metadata to output files in the output folder.

In [3]:
%run step0_get_neutral_baselines_intensity.py

Dynamic neutral arousal range: (np.float64(0.47), np.float64(0.49))
Processing file: /Users/rmerx/projects/naomi-ICL/0.0_corpus_preprocessing/output/natural_lines_targets/abuse.lines.psych
Saved neutral arousal sentences for abuse to /Users/rmerx/projects/naomi-ICL/3_intensity/synthetic/input/baselines/abuse_neutral_baselines_intensity.csv.
Total neutral sentences for abuse: 4262
Processing file: /Users/rmerx/projects/naomi-ICL/0.0_corpus_preprocessing/output/natural_lines_targets/anxiety.lines.psych
Saved neutral arousal sentences for anxiety to /Users/rmerx/projects/naomi-ICL/3_intensity/synthetic/input/baselines/anxiety_neutral_baselines_intensity.csv.
Total neutral sentences for anxiety: 12318
Processing file: /Users/rmerx/projects/naomi-ICL/0.0_corpus_preprocessing/output/natural_lines_targets/depression.lines.psych
Saved neutral arousal sentences for depression to /Users/rmerx/projects/naomi-ICL/3_intensity/synthetic/input/baselines/depression_neutral_baselines_intensity.csv.
Tot

Setup examples to inject in the prompt

In [8]:
# This code sets up examples of baseline sentences and their intensity-modified variations (more and less intense) to provide context and guidance for the LLM. 
# These examples are formatted into a structured prompt to help the model understand how to generate intensity-modified variations for new sentences.

@dataclass
class Example:
    baseline: str
    more_intense: str
    less_intense: str

    def format_for_prompt(self):
        return f"""<baseline>
{self.baseline}
</baseline>
<increased {TARGET_WORD_HUMAN} intensity>
{self.more_intense}
</increased {TARGET_WORD_HUMAN} intensity>
<decreased {TARGET_WORD_HUMAN} intensity>
{self.less_intense}
</decreased {TARGET_WORD_HUMAN} intensity>
"""

    @staticmethod
    def read_example_data(word: TARGET_WORD_CHOICES) -> pd.DataFrame:
        filepath = os.path.join('input', f'intensity_example_sentences.xlsx')
        # read to a dataframe
        df = pd.read_excel(filepath)
        # filter rows where `target` column is equal to the `word` argument
        df = df[df['target'] == word]
        print(f"Loaded {len(df)} examples for the term '{word}'")
        return df

example_data = Example.read_example_data(TARGET_WORD)

EXAMPLES = [
    Example(
        baseline=row['baseline'],
        more_intense=row['high_intensity'],
        less_intense=row['low_intensity']
    )
    for _, row in example_data.iterrows()
]

PROMPT_INTRO = f"""In psychology lexicographic research, we define "intensity" as the "extent to which a word refers to more emotionally or referentially intense phenomena". Here we study the intensity of the word "{TARGET_WORD_HUMAN}"
You will be given a sentence with the word "{TARGET_WORD_HUMAN}" in it. You will then be asked to write two new sentences: one where the word "{TARGET_WORD_HUMAN}" is more intense, and one where it is less intense.

"""


Loaded 5 examples for the term 'mental_illness'


In [20]:
# This code chunk constructs a structured prompt by combining predefined examples with a new target sentence, queries an LLM to generate intensity-modified variations (more and less intense) 
# of the target sentence, and extracts the results for further analysis.

@dataclass
class SentenceToModify:
    text: str
    increased_variation: str = None
    decreased_variation: str = None

    def get_prompt(self):
        prompt = PROMPT_INTRO
        for example in EXAMPLES:
            prompt += example.format_for_prompt()
            prompt += "\n\n"
        
        prompt += f"""<baseline>
{self.text}
</baseline>
"""
        return prompt
    
    def parse_response(self, response: str):
        # get the sentences inside <more {TARGET_WORD}> and <less {TARGET_WORD}>
        self.increased_variation = response.split(f"<increased {TARGET_WORD_HUMAN} intensity>")[1].split(f"</increased {TARGET_WORD_HUMAN} intensity>")[0].strip()
        self.decreased_variation = response.split(f"<decreased {TARGET_WORD_HUMAN} intensity>")[1].split(f"</decreased {TARGET_WORD_HUMAN} intensity>")[0].strip()
        return self.increased_variation, self.decreased_variation

    def get_variations(self) -> list[str]:
        """ Returns a list of two strings: one where the TARGET_WORD is more intense, and one where it is less intense """
        assert TARGET_WORD_HUMAN in self.text, f"word {TARGET_WORD_HUMAN} not found in {self.text}"
        prompt = self.get_prompt()
        res = sm.generate_text(prompt=prompt, llm_provider=PROVIDER, llm_model=MODEL)
        return self.parse_response(res)

    @staticmethod
    def load_baselines(word: TARGET_WORD_CHOICES) -> List[str]:
        # find the baselines.csv file in the `input` folder
        filepath = os.path.join('input', 'baselines', f'{word}_neutral_baselines_intensity.csv')
        df = pd.read_csv(filepath)
        # return the `sentence` column as a list
        print(f"Loaded a total of {len(df)} baseline sentences, sampling {MAX_BASELINES}")
        baselines = df['sentence'].tolist()
        baselines = random.sample(baselines, MAX_BASELINES) if len(baselines) > MAX_BASELINES else baselines
        baselines = [s.replace(TARGET_WORD, TARGET_WORD_HUMAN) for s in baselines]
        return baselines
    
    @staticmethod
    def save_sentences(sentences: List['SentenceToModify']):
        output_file = os.path.join('output', f'{TARGET_WORD}_synthetic_sentences.csv')
        df = pd.DataFrame([{'baseline': s.text, 'high_intensity': s.increased_variation, 'low_intensity': s.decreased_variation} for s in sentences])
        df.to_csv(output_file, index=False)
        print(f"Saved {len(sentences)} sentences to {output_file}")

baselines = SentenceToModify.load_baselines(TARGET_WORD)
sentence = SentenceToModify(text=baselines[0])
print(sentence.get_prompt())

Loaded a total of 680 baseline sentences, sampling 10
In psychology lexicographic research, we define "intensity" as the "extent to which a word refers to more emotionally or referentially intense phenomena". Here we study the intensity of the word "mental illness"
You will be given a sentence with the word "mental illness" in it. You will then be asked to write two new sentences: one where the word "mental illness" is more intense, and one where it is less intense.

<baseline>
The study examined the prevalence of mental illness in urban populations.
</baseline>
<increased mental illness intensity>
The study highlighted the staggering prevalence of severe mental illness in urban populations.
</increased mental illness intensity>
<decreased mental illness intensity>
The study explored the relatively low prevalence of mild mental illness in urban populations.
</decreased mental illness intensity>


<baseline>
The research analyzed the treatment of mental illness in clinical settings.
</b

In [18]:
sentences = []

for baseline in tqdm(baselines):
    sentence = SentenceToModify(text=baseline)
    positive_variation, negative_variation = sentence.get_variations()
    sentences.append(sentence)

SentenceToModify.save_sentences(sentences)


100%|██████████| 10/10 [00:24<00:00,  2.45s/it]

Saved 10 sentences to output/mental_illness_synthetic_sentences.csv
